In [1]:
import pandas as pd
import os
import glob
from datetime import datetime

In [2]:
path = os.getcwd() + "\\Dataset\\"
files = glob.glob(path + "/*precios*")

In [96]:
df_precios = pd.read_excel(files[0], dtype=object ,sheet_name=1)
df_precios1 = pd.read_excel(files[0], sheet_name=0)
df_precios2 = pd.read_csv(files[1],sep=',',encoding='utf-16')
df_precios3 = pd.read_json(files[2])

In [155]:
def unDate(x):
    """read_excel me trae algunas sucursal_id como si fueran datetime. Esta funcion toma elementos, chequea si son datetime y en caso de serlo, los convierte a str con el formato acorde al orden de identificadores de sucursal"""
    if isinstance(x,datetime):
        x=x.strftime("%d-%m-%Y")
    return x

def nasDups(df):
    """Cosas básicas: remover duplicados y nulos"""
    df = df.copy()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

def typeCheck(df):
    if df.precio.dtype != "float":
        try:
            df.precio = df.precio.astype(float)
        except:
            print("Error 1")
    if df.sucursal_id.dtype != "str":
        try:
            df.sucursal_id = df.sucursal_id.astype('string')
        except:
            print('Error str')
    return df

def splitProd(df):
    df.producto_id = df.producto_id.str.split('-').str[-1]
    return df


In [156]:
df_precios = nasDups(df_precios)
df_precios = splitProd(df_precios)
df_precios = typeCheck(df_precios)

In [158]:
def etl(df):
    df = nasDups(df)
    df = splitProd(df)
    df = typeCheck(df)
    return df

In [154]:
df_precios.producto_id.astype('string')

11125     2300006000003
11126     2300049000008
11127     2300073000005
11128     2300073000005
11129     2300075000003
              ...      
458382    0000000992855
458383    0000000992855
458384    0000000995696
458385    0000000997096
458386    0000000999137
Name: producto_id, Length: 12464, dtype: string

In [134]:
df_precios1[df_precios1.precio == df_precios1.precio.max()]

,precio,producto_id,sucursal_id
22476,4.399890e+16,NaN,36-3-25
22488,4.399890e+16,NaN,36-3-25
22489,4.399890e+16,NaN,36-3-25
22783,4.399890e+16,NaN,36-5-73


In [116]:
df_precios.precio.astype(float)

11125     829.00
11126      59.00
11127      59.00
11128       7.49
11129      59.00
           ...  
458382    145.00
458383    149.00
458384    509.00
458385     90.99
458386    349.99
Name: precio, Length: 12464, dtype: float64

Algunos precios son un str vacio: ''. Los convertimos a nulos para poder remover esos registros

In [86]:
df_precios3[df_precios3.precio == '']

,precio,producto_id,sucursal_id


In [87]:
df_precios3.precio = df_precios3.precio.apply(lambda x: None if x == '' else x)

In [77]:
df_precios.sucursal_id = df_precios['sucursal_id'].apply(unDate)

Vemos que hay nulos en precio y en producto_id.  
Los producto_id están como float, vamos a convertirlos

In [79]:
df_precios.producto_id =  df_precios['producto_id'].apply(lambda x: int(x) if isinstance(x,float) else None)
# df_precios.precio = df_precios.precio.astype(float)

In [89]:
df_precios2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472131 entries, 0 to 472165
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       472131 non-null  float64
 1   producto_id  472131 non-null  object 
 2   sucursal_id  472131 non-null  object 
dtypes: float64(1), object(2)
memory usage: 30.5+ MB


In [6]:
df_precios2 = nasDups(df_precios2)

In [39]:
df_precios.precio = df_precios.precio.astype(float)

In [129]:
df_precios2 = preProc(df_precios2)

In [144]:
count = 0
count_str = 0
for item in df_precios.producto_id:
    if isinstance(item,str):
        if '-' in item:
            count_str +=1
    else:
        count+=1
